# Download fasta sequences
Go to www.ensembl.org to find the gene id for the gene name TMPRSS2 and ERG

In [1]:
#TMPRSS2 
curl 'https://rest.ensembl.org/sequence/id/ENSG00000184012?type=cdna;multiple_sequences=100' -H 'Content-type:text/x-fasta' > TMPRSS2.fasta

#ERG
curl 'https://rest.ensembl.org/sequence/id/ENSG00000157554?type=cdna;multiple_sequences=100' -H 'Content-type:text/x-fasta' > ERG.fasta

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 52571  100 52571    0     0   107k      0 --:--:-- --:--:-- --:--:--  107k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 50291  100 50291    0     0  97652      0 --:--:-- --:--:-- --:--:-- 97463


: 1

## Manually create the fusions
[Cosmic Gene Fusion](https://cancer.sanger.ac.uk/cosmic/fusion/overview?fid=250888&gid=291726) to find breakpoints <br>
TMPRSS2_ENST00000332149, ERG_ENST00000442448 transcripts used for fusion <br>
- exon 2 from TMPRSS2 and end of exon 4 from ERG
- exon 3 from TMPRSS2 and beginning of exon 4 from ERG


# Create input fasta with defined wild type and fusion expression

In [1]:
mkdir WildType_Fusion_Quantify

: 1

In [12]:
# low wild type expression high fusion expression
for i in {1..1};do cat TMPRSS2_ENST00000332149.fasta; done > T_TMP.fasta
for i in {1..1};do cat ERG_ENST00000442448.fasta; done > E_TMP.fasta
for i in {1..4};do cat TMPRSS2_ERG_ManuallyFused.fasta; done > F_TMP.fasta
cat T_TMP.fasta E_TMP.fasta F_TMP.fasta >  WildType_Fusion_Quantify/LowWT_HighFusion.fasta
rm T_TMP.fasta E_TMP.fasta F_TMP.fasta

# low wild type expression high fusion expression
for i in {1..4};do cat TMPRSS2_ENST00000332149.fasta; done > T_TMP.fasta
for i in {1..4};do cat ERG_ENST00000442448.fasta; done > E_TMP.fasta
for i in {1..1};do cat TMPRSS2_ERG_ManuallyFused.fasta; done > F_TMP.fasta
cat T_TMP.fasta E_TMP.fasta F_TMP.fasta >  WildType_Fusion_Quantify/HighWT_LowFusion.fasta
rm T_TMP.fasta E_TMP.fasta F_TMP.fasta

# similar wild type expression and  fusion expression
for i in {1..3};do cat TMPRSS2_ENST00000332149.fasta; done > T_TMP.fasta
for i in {1..3};do cat ERG_ENST00000442448.fasta; done > E_TMP.fasta
for i in {1..2};do cat TMPRSS2_ERG_ManuallyFused.fasta; done > F_TMP.fasta
cat T_TMP.fasta E_TMP.fasta F_TMP.fasta >  WildType_Fusion_Quantify/SimilarWT_Fusion.fasta
rm T_TMP.fasta E_TMP.fasta F_TMP.fasta

: 1

In [14]:
grep ">" WildType_Fusion_Quantify/LowWT_HighFusion.fasta
echo ""
grep ">" WildType_Fusion_Quantify/HighWT_LowFusion.fasta
echo ""
grep ">" WildType_Fusion_Quantify/SimilarWT_Fusion.fasta

>ENST00000332149.10
>ENST00000442448.5
>TMPRSS2-exon2_ERG-exon4
>TMPRSS2-exon3_ERG-exon4
>TMPRSS2-exon2_ERG-exon4
>TMPRSS2-exon3_ERG-exon4
>TMPRSS2-exon2_ERG-exon4
>TMPRSS2-exon3_ERG-exon4
>TMPRSS2-exon2_ERG-exon4
>TMPRSS2-exon3_ERG-exon4

>ENST00000332149.10
>ENST00000332149.10
>ENST00000332149.10
>ENST00000332149.10
>ENST00000442448.5
>ENST00000442448.5
>ENST00000442448.5
>ENST00000442448.5
>TMPRSS2-exon2_ERG-exon4
>TMPRSS2-exon3_ERG-exon4

>ENST00000332149.10
>ENST00000332149.10
>ENST00000332149.10
>ENST00000442448.5
>ENST00000442448.5
>ENST00000442448.5
>TMPRSS2-exon2_ERG-exon4
>TMPRSS2-exon3_ERG-exon4
>TMPRSS2-exon2_ERG-exon4
>TMPRSS2-exon3_ERG-exon4


: 1

# Simulate reads from FASTA seq
There are 10 transcripts in the input fasta representing 3 genes(2 wild type and 1 fusion).
A rough estimate is 10000 reads per transcript, if an rnaseq experiment is 200M reads for 20,000 transcripts.
Hence art simulates 3000 reads from each transcript resulting in 30,000 reads in total

In [25]:
art=/home/jp/anaconda3/envs/jpEnv/bin/art_illumina

$art -sam -i WildType_Fusion_Quantify/HighWT_LowFusion.fasta -p -l 150 -ss HS25 -c 3000 -m 200 -s 10 -rs 3 -o HighWT_LowFusion
$art -sam -i WildType_Fusion_Quantify/LowWT_HighFusion.fasta -p -l 150 -ss HS25 -c 3000 -m 200 -s 10 -rs 3 -o LowWT_HighFusion
$art -sam -i WildType_Fusion_Quantify/SimilarWT_Fusion.fasta -p -l 150 -ss HS25 -c 3000 -m 200 -s 10 -rs 3 -o SimilarWT_Fusion


rm *.sam *.aln

mv *.fq WildType_Fusion_Quantify

=======================================ART======================================
                            ART_Illumina (2008-2015)                            
                            Q Version 2.3.7 (Mar 19, 2015)                      
                     Contact: Weichun Huang at whduke@gmail.com                 

                          Paired-end sequencing simulation

Total CPU time used: 1.01

The random seed for the run: 3

Parameters used during run
	Read Length:	150
	Genome masking 'N' cutoff frequency: 	1 in 150
	Fold Coverage:            0X
	Mean Fragment Length:     200
	Standard Deviation:       10
	Profile Type:             Combined
	ID Tag:                   

Quality Profile(s)
	First Read:   HiSeq 2500 Length 150 R1 (built-in profile) 
	First Read:   HiSeq 2500 Length 150 R2 (built-in profile) 
Output files

  FASTQ Sequence Files:
	 the 1st reads: HighWT_LowFusion1.fq
	 the 2nd reads: HighWT_LowFusion2.fq

  ALN Alignment Files:
	 the 1st reads: HighWT_LowFus

: 1

In [27]:
wc -l WildType_Fusion_Quantify/HighWT_LowFusion1.fq

120000 WildType_Fusion_Quantify/HighWT_LowFusion1.fq


: 1

# Downsampling FASTQ files

In [30]:
seqtk=/home/jp/anaconda3/envs/jpEnv/bin/seqtk

: 1

In [32]:
# downsample 50%
$seqtk sample -s3 WildType_Fusion_Quantify/HighWT_LowFusion1.fq 0.5 > WildType_Fusion_Quantify/HighWT_LowFusion1_0.5.fq
$seqtk sample -s3 WildType_Fusion_Quantify/HighWT_LowFusion2.fq 0.5 > WildType_Fusion_Quantify/HighWT_LowFusion2_0.5.fq

$seqtk sample -s3 WildType_Fusion_Quantify/LowWT_HighFusion1.fq 0.5 > WildType_Fusion_Quantify/LowWT_HighFusion1_0.5.fq
$seqtk sample -s3 WildType_Fusion_Quantify/LowWT_HighFusion2.fq 0.5 > WildType_Fusion_Quantify/LowWT_HighFusion2_0.5.fq

$seqtk sample -s3 WildType_Fusion_Quantify/SimilarWT_Fusion1.fq 0.5 > WildType_Fusion_Quantify/SimilarWT_Fusion1_0.5.fq
$seqtk sample -s3 WildType_Fusion_Quantify/SimilarWT_Fusion2.fq 0.5 > WildType_Fusion_Quantify/SimilarWT_Fusion2_0.5.fq



# downsample 40%
$seqtk sample -s3 WildType_Fusion_Quantify/HighWT_LowFusion1.fq 0.4 > WildType_Fusion_Quantify/HighWT_LowFusion1_0.4.fq
$seqtk sample -s3 WildType_Fusion_Quantify/HighWT_LowFusion2.fq 0.4 > WildType_Fusion_Quantify/HighWT_LowFusion2_0.4.fq

$seqtk sample -s3 WildType_Fusion_Quantify/LowWT_HighFusion1.fq 0.4 > WildType_Fusion_Quantify/LowWT_HighFusion1_0.4.fq
$seqtk sample -s3 WildType_Fusion_Quantify/LowWT_HighFusion2.fq 0.4 > WildType_Fusion_Quantify/LowWT_HighFusion2_0.4.fq

$seqtk sample -s3 WildType_Fusion_Quantify/SimilarWT_Fusion1.fq 0.4 > WildType_Fusion_Quantify/SimilarWT_Fusion1_0.4.fq
$seqtk sample -s3 WildType_Fusion_Quantify/SimilarWT_Fusion2.fq 0.4 > WildType_Fusion_Quantify/SimilarWT_Fusion2_0.4.fq



# downsample 30%
$seqtk sample -s3 WildType_Fusion_Quantify/HighWT_LowFusion1.fq 0.3 > WildType_Fusion_Quantify/HighWT_LowFusion1_0.3.fq
$seqtk sample -s3 WildType_Fusion_Quantify/HighWT_LowFusion2.fq 0.3 > WildType_Fusion_Quantify/HighWT_LowFusion2_0.3.fq

$seqtk sample -s3 WildType_Fusion_Quantify/LowWT_HighFusion1.fq 0.3 > WildType_Fusion_Quantify/LowWT_HighFusion1_0.3.fq
$seqtk sample -s3 WildType_Fusion_Quantify/LowWT_HighFusion2.fq 0.3 > WildType_Fusion_Quantify/LowWT_HighFusion2_0.3.fq

$seqtk sample -s3 WildType_Fusion_Quantify/SimilarWT_Fusion1.fq 0.3 > WildType_Fusion_Quantify/SimilarWT_Fusion1_0.3.fq
$seqtk sample -s3 WildType_Fusion_Quantify/SimilarWT_Fusion2.fq 0.3 > WildType_Fusion_Quantify/SimilarWT_Fusion2_0.3.fq




# downsample 20%
$seqtk sample -s3 WildType_Fusion_Quantify/HighWT_LowFusion1.fq 0.2 > WildType_Fusion_Quantify/HighWT_LowFusion1_0.2.fq
$seqtk sample -s3 WildType_Fusion_Quantify/HighWT_LowFusion2.fq 0.2 > WildType_Fusion_Quantify/HighWT_LowFusion2_0.2.fq

$seqtk sample -s3 WildType_Fusion_Quantify/LowWT_HighFusion1.fq 0.2 > WildType_Fusion_Quantify/LowWT_HighFusion1_0.2.fq
$seqtk sample -s3 WildType_Fusion_Quantify/LowWT_HighFusion2.fq 0.2 > WildType_Fusion_Quantify/LowWT_HighFusion2_0.2.fq

$seqtk sample -s3 WildType_Fusion_Quantify/SimilarWT_Fusion1.fq 0.2 > WildType_Fusion_Quantify/SimilarWT_Fusion1_0.2.fq
$seqtk sample -s3 WildType_Fusion_Quantify/SimilarWT_Fusion2.fq 0.2 > WildType_Fusion_Quantify/SimilarWT_Fusion2_0.2.fq



# downsample 10%
$seqtk sample -s3 WildType_Fusion_Quantify/HighWT_LowFusion1.fq 0.1 > WildType_Fusion_Quantify/HighWT_LowFusion1_0.1.fq
$seqtk sample -s3 WildType_Fusion_Quantify/HighWT_LowFusion2.fq 0.1 > WildType_Fusion_Quantify/HighWT_LowFusion2_0.1.fq

$seqtk sample -s3 WildType_Fusion_Quantify/LowWT_HighFusion1.fq 0.1 > WildType_Fusion_Quantify/LowWT_HighFusion1_0.1.fq
$seqtk sample -s3 WildType_Fusion_Quantify/LowWT_HighFusion2.fq 0.1 > WildType_Fusion_Quantify/LowWT_HighFusion2_0.1.fq

$seqtk sample -s3 WildType_Fusion_Quantify/SimilarWT_Fusion1.fq 0.1 > WildType_Fusion_Quantify/SimilarWT_Fusion1_0.1.fq
$seqtk sample -s3 WildType_Fusion_Quantify/SimilarWT_Fusion2.fq 0.1 > WildType_Fusion_Quantify/SimilarWT_Fusion2_0.1.fq



: 1

In [34]:
wc -l WildType_Fusion_Quantify/LowWT_HighFusion1_0.1.fq

12160 WildType_Fusion_Quantify/LowWT_HighFusion1_0.1.fq


: 1

# Run STAR-Fusion

In [36]:
mkdir StarFusion

: 1

In [59]:
gen=/home/jp/STAR-Fusion-v1.10.0/GRCh38_gencode_v37_CTAT_lib_Mar012021.plug-n-play/ctat_genome_lib_build_dir
starFusion=/home/jp/STAR-Fusion-v1.10.0/STAR-Fusion
thread=10

: 1

## High WT Low Fusion

In [61]:
p1=WildType_Fusion_Quantify/HighWT_LowFusion1.fq
p2=WildType_Fusion_Quantify/HighWT_LowFusion2.fq

$starFusion --left_fq $p1 \
--right_fq $p2 \
--genome_lib_dir $gen \
--output_dir WildType_Fusions_Results/HighWT_LowFusion --CPU $thread


# 50% downsampling
p1=WildType_Fusion_Quantify/HighWT_LowFusion1_0.5.fq
p2=WildType_Fusion_Quantify/HighWT_LowFusion2_0.5.fq

$starFusion --left_fq $p1 \
--right_fq $p2 \
--genome_lib_dir $gen \
--output_dir WildType_Fusions_Results/HighWT_LowFusion_0.5 --CPU $thread


# 40% downsampling
p1=WildType_Fusion_Quantify/HighWT_LowFusion1_0.4.fq
p2=WildType_Fusion_Quantify/HighWT_LowFusion2_0.4.fq

$starFusion --left_fq $p1 \
--right_fq $p2 \
--genome_lib_dir $gen \
--output_dir WildType_Fusions_Results/HighWT_LowFusion_0.4 --CPU $thread


# 30% downsampling
p1=WildType_Fusion_Quantify/HighWT_LowFusion1_0.3.fq
p2=WildType_Fusion_Quantify/HighWT_LowFusion2_0.3.fq

$starFusion --left_fq $p1 \
--right_fq $p2 \
--genome_lib_dir $gen \
--output_dir WildType_Fusions_Results/HighWT_LowFusion_0.3 --CPU $thread


# 20% downsampling
p1=WildType_Fusion_Quantify/HighWT_LowFusion1_0.2.fq
p2=WildType_Fusion_Quantify/HighWT_LowFusion2_0.2.fq

$starFusion --left_fq $p1 \
--right_fq $p2 \
--genome_lib_dir $gen \
--output_dir WildType_Fusions_Results/HighWT_LowFusion_0.2 --CPU $thread


# 10% downsampling
p1=WildType_Fusion_Quantify/HighWT_LowFusion1_0.1.fq
p2=WildType_Fusion_Quantify/HighWT_LowFusion2_0.1.fq

$starFusion --left_fq $p1 \
--right_fq $p2 \
--genome_lib_dir $gen \
--output_dir WildType_Fusions_Results/HighWT_LowFusion_0.1 --CPU $thread


-ctat genome lib [/home/jp/STAR-Fusion-v1.10.0/GRCh38_gencode_v37_CTAT_lib_Mar012021.plug-n-play/ctat_genome_lib_build_dir] validates.
-- Skipping CMD: /home/jp/anaconda3/bin/STAR --genomeDir /home/jp/STAR-Fusion-v1.10.0/GRCh38_gencode_v37_CTAT_lib_Mar012021.plug-n-play/ctat_genome_lib_build_dir/ref_genome.fa.star.idx  --outReadsUnmapped None  --chimSegmentMin 12  --chimJunctionOverhangMin 8  --chimOutJunctionFormat 1  --alignSJDBoverhangMin 10  --alignMatesGapMax 100000  --alignIntronMax 100000  --alignSJstitchMismatchNmax 5 -1 5 5  --runThreadN 10 --outSAMstrandField intronMotif  --outSAMunmapped Within  --alignInsertionFlush Right  --alignSplicedMateMapLminOverLmate 0  --alignSplicedMateMapLmin 30  --outSAMtype BAM Unsorted  --readFilesIn /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusion_Quantify/HighWT_LowFusion1.fq /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusion_Quantify/HighWT_LowFusion2.fq  --outSAMattrRGline ID:GRPundef  --chimMultimapScoreRange 3  --chimScoreJunctionNonGTA



-ctat genome lib [/home/jp/STAR-Fusion-v1.10.0/GRCh38_gencode_v37_CTAT_lib_Mar012021.plug-n-play/ctat_genome_lib_build_dir] validates.
CMD: mkdir -p /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/HighWT_LowFusion_0.5
CMD: mkdir -p /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/HighWT_LowFusion_0.5/_starF_checkpoints
CMD: mkdir -p /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/HighWT_LowFusion_0.5/star-fusion.preliminary
* Running CMD: /home/jp/anaconda3/bin/STAR --genomeDir /home/jp/STAR-Fusion-v1.10.0/GRCh38_gencode_v37_CTAT_lib_Mar012021.plug-n-play/ctat_genome_lib_build_dir/ref_genome.fa.star.idx  --outReadsUnmapped None  --chimSegmentMin 12  --chimJunctionOverhangMin 8  --chimOutJunctionFormat 1  --alignSJDBoverhangMin 10  --alignMatesGapMax 100000  --alignIntronMax 100000  --alignSJstitchMismatchNmax 5 -1 5 5  --runThreadN 10 --outSAMstrandField intronMotif  --outSAMunmapped Within  --alignInsertionFlush Right  --alignSplicedMateMapLminOver

CMD: /home/jp/STAR-Fusion-v1.10.0/util/column_exclusions.pl /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/HighWT_LowFusion_0.5/star-fusion.preliminary/star-fusion.filter.intermediates_dir/star-fusion.post_blast_and_promiscuity_filter JunctionReads,SpanningFrags > /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/HighWT_LowFusion_0.5/star-fusion.preliminary/star-fusion.filter.intermediates_dir/star-fusion.post_blast_and_promiscuity_filter.abridged
CMD: ln -sf /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/HighWT_LowFusion_0.5/star-fusion.preliminary/star-fusion.filter.intermediates_dir/star-fusion.post_blast_and_promiscuity_filter /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/HighWT_LowFusion_0.5/star-fusion.preliminary/star-fusion.fusion_candidates.preliminary.filtered
* Running CMD: /home/jp/STAR-Fusion-v1.10.0/util/incorporate_FFPM_into_final_report.pl /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/HighWT_LowFusion_0.5/s

-parsing /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/HighWT_LowFusion_0.4/star-fusion.preliminary/star-fusion.junction_breakpts_to_genes.txt, organizing read mapping info.

done.

* Running CMD: /home/jp/STAR-Fusion-v1.10.0/util/STAR-Fusion.predict  -J /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/HighWT_LowFusion_0.4/star-fusion.preliminary/star-fusion.junction_breakpts_to_genes.txt.pass  -O /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/HighWT_LowFusion_0.4/star-fusion.preliminary/star-fusion 
-parsing fusion evidence: /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/HighWT_LowFusion_0.4/star-fusion.preliminary/star-fusion.junction_breakpts_to_genes.txt.pass
-outputting fusion candidates to file: /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/HighWT_LowFusion_0.4/star-fusion.preliminary/star-fusion.fusion_candidates.preliminary
* Running CMD: /home/jp/STAR-Fusion-v1.10.0/util/STAR-Fusion.filter  --fusion_preds /home/j

CMD: mkdir -p /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/HighWT_LowFusion_0.3
CMD: mkdir -p /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/HighWT_LowFusion_0.3/_starF_checkpoints
CMD: mkdir -p /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/HighWT_LowFusion_0.3/star-fusion.preliminary
* Running CMD: /home/jp/anaconda3/bin/STAR --genomeDir /home/jp/STAR-Fusion-v1.10.0/GRCh38_gencode_v37_CTAT_lib_Mar012021.plug-n-play/ctat_genome_lib_build_dir/ref_genome.fa.star.idx  --outReadsUnmapped None  --chimSegmentMin 12  --chimJunctionOverhangMin 8  --chimOutJunctionFormat 1  --alignSJDBoverhangMin 10  --alignMatesGapMax 100000  --alignIntronMax 100000  --alignSJstitchMismatchNmax 5 -1 5 5  --runThreadN 10 --outSAMstrandField intronMotif  --outSAMunmapped Within  --alignInsertionFlush Right  --alignSplicedMateMapLminOverLmate 0  --alignSplicedMateMapLmin 30  --outSAMtype BAM Unsorted  --readFilesIn /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusion_Quanti

CMD: ln -sf /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/HighWT_LowFusion_0.3/star-fusion.preliminary/star-fusion.filter.intermediates_dir/star-fusion.post_blast_and_promiscuity_filter /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/HighWT_LowFusion_0.3/star-fusion.preliminary/star-fusion.fusion_candidates.preliminary.filtered
* Running CMD: /home/jp/STAR-Fusion-v1.10.0/util/incorporate_FFPM_into_final_report.pl /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/HighWT_LowFusion_0.3/star-fusion.preliminary/star-fusion.fusion_candidates.preliminary.filtered 9137 > /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/HighWT_LowFusion_0.3/star-fusion.preliminary/star-fusion.fusion_candidates.preliminary.filtered.FFPM
* Running CMD: /home/jp/STAR-Fusion-v1.10.0/util/append_breakpoint_junction_info.pl /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/HighWT_LowFusion_0.3/star-fusion.preliminary/star-fusion.fusion_candidates.preliminary.fi


done.

* Running CMD: /home/jp/STAR-Fusion-v1.10.0/util/STAR-Fusion.predict  -J /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/HighWT_LowFusion_0.2/star-fusion.preliminary/star-fusion.junction_breakpts_to_genes.txt.pass  -O /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/HighWT_LowFusion_0.2/star-fusion.preliminary/star-fusion 
-parsing fusion evidence: /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/HighWT_LowFusion_0.2/star-fusion.preliminary/star-fusion.junction_breakpts_to_genes.txt.pass
-outputting fusion candidates to file: /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/HighWT_LowFusion_0.2/star-fusion.preliminary/star-fusion.fusion_candidates.preliminary
* Running CMD: /home/jp/STAR-Fusion-v1.10.0/util/STAR-Fusion.filter  --fusion_preds /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/HighWT_LowFusion_0.2/star-fusion.preliminary/star-fusion.fusion_candidates.preliminary  -J Chimeric.out.junction  --tmpdir /tmp  --min_

CMD: mkdir -p /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/HighWT_LowFusion_0.1/star-fusion.preliminary
* Running CMD: /home/jp/anaconda3/bin/STAR --genomeDir /home/jp/STAR-Fusion-v1.10.0/GRCh38_gencode_v37_CTAT_lib_Mar012021.plug-n-play/ctat_genome_lib_build_dir/ref_genome.fa.star.idx  --outReadsUnmapped None  --chimSegmentMin 12  --chimJunctionOverhangMin 8  --chimOutJunctionFormat 1  --alignSJDBoverhangMin 10  --alignMatesGapMax 100000  --alignIntronMax 100000  --alignSJstitchMismatchNmax 5 -1 5 5  --runThreadN 10 --outSAMstrandField intronMotif  --outSAMunmapped Within  --alignInsertionFlush Right  --alignSplicedMateMapLminOverLmate 0  --alignSplicedMateMapLmin 30  --outSAMtype BAM Unsorted  --readFilesIn /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusion_Quantify/HighWT_LowFusion1_0.1.fq /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusion_Quantify/HighWT_LowFusion2_0.1.fq  --outSAMattrRGline ID:GRPundef  --chimMultimapScoreRange 3  --chimScoreJunctionNonGTAG -4  --ch

CMD: ln -sf /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/HighWT_LowFusion_0.1/star-fusion.preliminary/star-fusion.filter.intermediates_dir/star-fusion.post_blast_and_promiscuity_filter /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/HighWT_LowFusion_0.1/star-fusion.preliminary/star-fusion.fusion_candidates.preliminary.filtered
* Running CMD: /home/jp/STAR-Fusion-v1.10.0/util/incorporate_FFPM_into_final_report.pl /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/HighWT_LowFusion_0.1/star-fusion.preliminary/star-fusion.fusion_candidates.preliminary.filtered 3040 > /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/HighWT_LowFusion_0.1/star-fusion.preliminary/star-fusion.fusion_candidates.preliminary.filtered.FFPM
* Running CMD: /home/jp/STAR-Fusion-v1.10.0/util/append_breakpoint_junction_info.pl /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/HighWT_LowFusion_0.1/star-fusion.preliminary/star-fusion.fusion_candidates.preliminary.fi

: 1

## Low WT High Fusion

In [63]:
p1=WildType_Fusion_Quantify/LowWT_HighFusion1.fq
p2=WildType_Fusion_Quantify/LowWT_HighFusion2.fq

$starFusion --left_fq $p1 \
--right_fq $p2 \
--genome_lib_dir $gen \
--output_dir WildType_Fusions_Results/LowWT_HighFusion --CPU $thread


# 50% downsampling
p1=WildType_Fusion_Quantify/LowWT_HighFusion1_0.5.fq
p2=WildType_Fusion_Quantify/LowWT_HighFusion2_0.5.fq

$starFusion --left_fq $p1 \
--right_fq $p2 \
--genome_lib_dir $gen \
--output_dir WildType_Fusions_Results/LowWT_HighFusion_0.5 --CPU $thread


# 40% downsampling
p1=WildType_Fusion_Quantify/LowWT_HighFusion1_0.4.fq
p2=WildType_Fusion_Quantify/LowWT_HighFusion2_0.4.fq

$starFusion --left_fq $p1 \
--right_fq $p2 \
--genome_lib_dir $gen \
--output_dir WildType_Fusions_Results/LowWT_HighFusion_0.4 --CPU $thread


# 30% downsampling
p1=WildType_Fusion_Quantify/LowWT_HighFusion1_0.3.fq
p2=WildType_Fusion_Quantify/LowWT_HighFusion2_0.3.fq

$starFusion --left_fq $p1 \
--right_fq $p2 \
--genome_lib_dir $gen \
--output_dir WildType_Fusions_Results/LowWT_HighFusion_0.3 --CPU $thread


# 20% downsampling
p1=WildType_Fusion_Quantify/LowWT_HighFusion1_0.2.fq
p2=WildType_Fusion_Quantify/LowWT_HighFusion2_0.2.fq

$starFusion --left_fq $p1 \
--right_fq $p2 \
--genome_lib_dir $gen \
--output_dir WildType_Fusions_Results/LowWT_HighFusion_0.2 --CPU $thread


# 10% downsampling
p1=WildType_Fusion_Quantify/LowWT_HighFusion1_0.1.fq
p2=WildType_Fusion_Quantify/LowWT_HighFusion2_0.1.fq

$starFusion --left_fq $p1 \
--right_fq $p2 \
--genome_lib_dir $gen \
--output_dir WildType_Fusions_Results/LowWT_HighFusion_0.1 --CPU $thread


-ctat genome lib [/home/jp/STAR-Fusion-v1.10.0/GRCh38_gencode_v37_CTAT_lib_Mar012021.plug-n-play/ctat_genome_lib_build_dir] validates.
CMD: mkdir -p /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/LowWT_HighFusion
CMD: mkdir -p /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/LowWT_HighFusion/_starF_checkpoints
CMD: mkdir -p /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/LowWT_HighFusion/star-fusion.preliminary
* Running CMD: /home/jp/anaconda3/bin/STAR --genomeDir /home/jp/STAR-Fusion-v1.10.0/GRCh38_gencode_v37_CTAT_lib_Mar012021.plug-n-play/ctat_genome_lib_build_dir/ref_genome.fa.star.idx  --outReadsUnmapped None  --chimSegmentMin 12  --chimJunctionOverhangMin 8  --chimOutJunctionFormat 1  --alignSJDBoverhangMin 10  --alignMatesGapMax 100000  --alignIntronMax 100000  --alignSJstitchMismatchNmax 5 -1 5 5  --runThreadN 10 --outSAMstrandField intronMotif  --outSAMunmapped Within  --alignInsertionFlush Right  --alignSplicedMateMapLminOverLmate 0  --ali

CMD: ln -sf /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/LowWT_HighFusion/star-fusion.preliminary/star-fusion.filter.intermediates_dir/star-fusion.post_blast_and_promiscuity_filter /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/LowWT_HighFusion/star-fusion.preliminary/star-fusion.fusion_candidates.preliminary.filtered
* Running CMD: /home/jp/STAR-Fusion-v1.10.0/util/incorporate_FFPM_into_final_report.pl /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/LowWT_HighFusion/star-fusion.preliminary/star-fusion.fusion_candidates.preliminary.filtered 30000 > /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/LowWT_HighFusion/star-fusion.preliminary/star-fusion.fusion_candidates.preliminary.filtered.FFPM
* Running CMD: /home/jp/STAR-Fusion-v1.10.0/util/append_breakpoint_junction_info.pl /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/LowWT_HighFusion/star-fusion.preliminary/star-fusion.fusion_candidates.preliminary.filtered.FFPM /home/j


done.

* Running CMD: /home/jp/STAR-Fusion-v1.10.0/util/STAR-Fusion.predict  -J /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/LowWT_HighFusion_0.5/star-fusion.preliminary/star-fusion.junction_breakpts_to_genes.txt.pass  -O /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/LowWT_HighFusion_0.5/star-fusion.preliminary/star-fusion 
-parsing fusion evidence: /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/LowWT_HighFusion_0.5/star-fusion.preliminary/star-fusion.junction_breakpts_to_genes.txt.pass
-outputting fusion candidates to file: /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/LowWT_HighFusion_0.5/star-fusion.preliminary/star-fusion.fusion_candidates.preliminary
* Running CMD: /home/jp/STAR-Fusion-v1.10.0/util/STAR-Fusion.filter  --fusion_preds /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/LowWT_HighFusion_0.5/star-fusion.preliminary/star-fusion.fusion_candidates.preliminary  -J Chimeric.out.junction  --tmpdir /tmp  --min_

CMD: mkdir -p /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/LowWT_HighFusion_0.4/star-fusion.preliminary
* Running CMD: /home/jp/anaconda3/bin/STAR --genomeDir /home/jp/STAR-Fusion-v1.10.0/GRCh38_gencode_v37_CTAT_lib_Mar012021.plug-n-play/ctat_genome_lib_build_dir/ref_genome.fa.star.idx  --outReadsUnmapped None  --chimSegmentMin 12  --chimJunctionOverhangMin 8  --chimOutJunctionFormat 1  --alignSJDBoverhangMin 10  --alignMatesGapMax 100000  --alignIntronMax 100000  --alignSJstitchMismatchNmax 5 -1 5 5  --runThreadN 10 --outSAMstrandField intronMotif  --outSAMunmapped Within  --alignInsertionFlush Right  --alignSplicedMateMapLminOverLmate 0  --alignSplicedMateMapLmin 30  --outSAMtype BAM Unsorted  --readFilesIn /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusion_Quantify/LowWT_HighFusion1_0.4.fq /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusion_Quantify/LowWT_HighFusion2_0.4.fq  --outSAMattrRGline ID:GRPundef  --chimMultimapScoreRange 3  --chimScoreJunctionNonGTAG -4  --ch

CMD: ln -sf /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/LowWT_HighFusion_0.4/star-fusion.preliminary/star-fusion.filter.intermediates_dir/star-fusion.post_blast_and_promiscuity_filter /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/LowWT_HighFusion_0.4/star-fusion.preliminary/star-fusion.fusion_candidates.preliminary.filtered
* Running CMD: /home/jp/STAR-Fusion-v1.10.0/util/incorporate_FFPM_into_final_report.pl /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/LowWT_HighFusion_0.4/star-fusion.preliminary/star-fusion.fusion_candidates.preliminary.filtered 12120 > /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/LowWT_HighFusion_0.4/star-fusion.preliminary/star-fusion.fusion_candidates.preliminary.filtered.FFPM
* Running CMD: /home/jp/STAR-Fusion-v1.10.0/util/append_breakpoint_junction_info.pl /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/LowWT_HighFusion_0.4/star-fusion.preliminary/star-fusion.fusion_candidates.preliminary.f


done.

* Running CMD: /home/jp/STAR-Fusion-v1.10.0/util/STAR-Fusion.predict  -J /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/LowWT_HighFusion_0.3/star-fusion.preliminary/star-fusion.junction_breakpts_to_genes.txt.pass  -O /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/LowWT_HighFusion_0.3/star-fusion.preliminary/star-fusion 
-parsing fusion evidence: /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/LowWT_HighFusion_0.3/star-fusion.preliminary/star-fusion.junction_breakpts_to_genes.txt.pass
-outputting fusion candidates to file: /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/LowWT_HighFusion_0.3/star-fusion.preliminary/star-fusion.fusion_candidates.preliminary
* Running CMD: /home/jp/STAR-Fusion-v1.10.0/util/STAR-Fusion.filter  --fusion_preds /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/LowWT_HighFusion_0.3/star-fusion.preliminary/star-fusion.fusion_candidates.preliminary  -J Chimeric.out.junction  --tmpdir /tmp  --min_

CMD: mkdir -p /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/LowWT_HighFusion_0.2/star-fusion.preliminary
* Running CMD: /home/jp/anaconda3/bin/STAR --genomeDir /home/jp/STAR-Fusion-v1.10.0/GRCh38_gencode_v37_CTAT_lib_Mar012021.plug-n-play/ctat_genome_lib_build_dir/ref_genome.fa.star.idx  --outReadsUnmapped None  --chimSegmentMin 12  --chimJunctionOverhangMin 8  --chimOutJunctionFormat 1  --alignSJDBoverhangMin 10  --alignMatesGapMax 100000  --alignIntronMax 100000  --alignSJstitchMismatchNmax 5 -1 5 5  --runThreadN 10 --outSAMstrandField intronMotif  --outSAMunmapped Within  --alignInsertionFlush Right  --alignSplicedMateMapLminOverLmate 0  --alignSplicedMateMapLmin 30  --outSAMtype BAM Unsorted  --readFilesIn /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusion_Quantify/LowWT_HighFusion1_0.2.fq /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusion_Quantify/LowWT_HighFusion2_0.2.fq  --outSAMattrRGline ID:GRPundef  --chimMultimapScoreRange 3  --chimScoreJunctionNonGTAG -4  --ch

CMD: ln -sf /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/LowWT_HighFusion_0.2/star-fusion.preliminary/star-fusion.filter.intermediates_dir/star-fusion.post_blast_and_promiscuity_filter /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/LowWT_HighFusion_0.2/star-fusion.preliminary/star-fusion.fusion_candidates.preliminary.filtered
* Running CMD: /home/jp/STAR-Fusion-v1.10.0/util/incorporate_FFPM_into_final_report.pl /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/LowWT_HighFusion_0.2/star-fusion.preliminary/star-fusion.fusion_candidates.preliminary.filtered 6113 > /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/LowWT_HighFusion_0.2/star-fusion.preliminary/star-fusion.fusion_candidates.preliminary.filtered.FFPM
* Running CMD: /home/jp/STAR-Fusion-v1.10.0/util/append_breakpoint_junction_info.pl /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/LowWT_HighFusion_0.2/star-fusion.preliminary/star-fusion.fusion_candidates.preliminary.fi


done.

* Running CMD: /home/jp/STAR-Fusion-v1.10.0/util/STAR-Fusion.predict  -J /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/LowWT_HighFusion_0.1/star-fusion.preliminary/star-fusion.junction_breakpts_to_genes.txt.pass  -O /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/LowWT_HighFusion_0.1/star-fusion.preliminary/star-fusion 
-parsing fusion evidence: /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/LowWT_HighFusion_0.1/star-fusion.preliminary/star-fusion.junction_breakpts_to_genes.txt.pass
-outputting fusion candidates to file: /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/LowWT_HighFusion_0.1/star-fusion.preliminary/star-fusion.fusion_candidates.preliminary
* Running CMD: /home/jp/STAR-Fusion-v1.10.0/util/STAR-Fusion.filter  --fusion_preds /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/LowWT_HighFusion_0.1/star-fusion.preliminary/star-fusion.fusion_candidates.preliminary  -J Chimeric.out.junction  --tmpdir /tmp  --min_

: 1

## Similar WT and Fusion

In [64]:
p1=WildType_Fusion_Quantify/SimilarWT_Fusion1.fq
p2=WildType_Fusion_Quantify/SimilarWT_Fusion2.fq

$starFusion --left_fq $p1 \
--right_fq $p2 \
--genome_lib_dir $gen \
--output_dir WildType_Fusions_Results/SimilarWT_Fusion --CPU $thread


# 50% downsampling
p1=WildType_Fusion_Quantify/SimilarWT_Fusion1_0.5.fq
p2=WildType_Fusion_Quantify/SimilarWT_Fusion2_0.5.fq

$starFusion --left_fq $p1 \
--right_fq $p2 \
--genome_lib_dir $gen \
--output_dir WildType_Fusions_Results/SimilarWT_Fusion_0.5 --CPU $thread


# 40% downsampling
p1=WildType_Fusion_Quantify/SimilarWT_Fusion1_0.4.fq
p2=WildType_Fusion_Quantify/SimilarWT_Fusion2_0.4.fq

$starFusion --left_fq $p1 \
--right_fq $p2 \
--genome_lib_dir $gen \
--output_dir WildType_Fusions_Results/SimilarWT_Fusion_0.4 --CPU $thread


# 30% downsampling
p1=WildType_Fusion_Quantify/SimilarWT_Fusion1_0.3.fq
p2=WildType_Fusion_Quantify/SimilarWT_Fusion2_0.3.fq

$starFusion --left_fq $p1 \
--right_fq $p2 \
--genome_lib_dir $gen \
--output_dir WildType_Fusions_Results/SimilarWT_Fusion_0.3 --CPU $thread


# 20% downsampling
p1=WildType_Fusion_Quantify/SimilarWT_Fusion1_0.2.fq
p2=WildType_Fusion_Quantify/SimilarWT_Fusion2_0.2.fq

$starFusion --left_fq $p1 \
--right_fq $p2 \
--genome_lib_dir $gen \
--output_dir WildType_Fusions_Results/SimilarWT_Fusion_0.2 --CPU $thread


# 10% downsampling
p1=WildType_Fusion_Quantify/SimilarWT_Fusion1_0.1.fq
p2=WildType_Fusion_Quantify/SimilarWT_Fusion2_0.1.fq

$starFusion --left_fq $p1 \
--right_fq $p2 \
--genome_lib_dir $gen \
--output_dir WildType_Fusions_Results/SimilarWT_Fusion_0.1 --CPU $thread


-ctat genome lib [/home/jp/STAR-Fusion-v1.10.0/GRCh38_gencode_v37_CTAT_lib_Mar012021.plug-n-play/ctat_genome_lib_build_dir] validates.
CMD: mkdir -p /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/SimilarWT_Fusion
CMD: mkdir -p /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/SimilarWT_Fusion/_starF_checkpoints
CMD: mkdir -p /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/SimilarWT_Fusion/star-fusion.preliminary
* Running CMD: /home/jp/anaconda3/bin/STAR --genomeDir /home/jp/STAR-Fusion-v1.10.0/GRCh38_gencode_v37_CTAT_lib_Mar012021.plug-n-play/ctat_genome_lib_build_dir/ref_genome.fa.star.idx  --outReadsUnmapped None  --chimSegmentMin 12  --chimJunctionOverhangMin 8  --chimOutJunctionFormat 1  --alignSJDBoverhangMin 10  --alignMatesGapMax 100000  --alignIntronMax 100000  --alignSJstitchMismatchNmax 5 -1 5 5  --runThreadN 10 --outSAMstrandField intronMotif  --outSAMunmapped Within  --alignInsertionFlush Right  --alignSplicedMateMapLminOverLmate 0  --ali

CMD: ln -sf /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/SimilarWT_Fusion/star-fusion.preliminary/star-fusion.filter.intermediates_dir/star-fusion.post_blast_and_promiscuity_filter /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/SimilarWT_Fusion/star-fusion.preliminary/star-fusion.fusion_candidates.preliminary.filtered
* Running CMD: /home/jp/STAR-Fusion-v1.10.0/util/incorporate_FFPM_into_final_report.pl /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/SimilarWT_Fusion/star-fusion.preliminary/star-fusion.fusion_candidates.preliminary.filtered 30000 > /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/SimilarWT_Fusion/star-fusion.preliminary/star-fusion.fusion_candidates.preliminary.filtered.FFPM
* Running CMD: /home/jp/STAR-Fusion-v1.10.0/util/append_breakpoint_junction_info.pl /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/SimilarWT_Fusion/star-fusion.preliminary/star-fusion.fusion_candidates.preliminary.filtered.FFPM /home/j


done.

* Running CMD: /home/jp/STAR-Fusion-v1.10.0/util/STAR-Fusion.predict  -J /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/SimilarWT_Fusion_0.5/star-fusion.preliminary/star-fusion.junction_breakpts_to_genes.txt.pass  -O /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/SimilarWT_Fusion_0.5/star-fusion.preliminary/star-fusion 
-parsing fusion evidence: /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/SimilarWT_Fusion_0.5/star-fusion.preliminary/star-fusion.junction_breakpts_to_genes.txt.pass
-outputting fusion candidates to file: /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/SimilarWT_Fusion_0.5/star-fusion.preliminary/star-fusion.fusion_candidates.preliminary
* Running CMD: /home/jp/STAR-Fusion-v1.10.0/util/STAR-Fusion.filter  --fusion_preds /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/SimilarWT_Fusion_0.5/star-fusion.preliminary/star-fusion.fusion_candidates.preliminary  -J Chimeric.out.junction  --tmpdir /tmp  --min_

CMD: mkdir -p /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/SimilarWT_Fusion_0.4/star-fusion.preliminary
* Running CMD: /home/jp/anaconda3/bin/STAR --genomeDir /home/jp/STAR-Fusion-v1.10.0/GRCh38_gencode_v37_CTAT_lib_Mar012021.plug-n-play/ctat_genome_lib_build_dir/ref_genome.fa.star.idx  --outReadsUnmapped None  --chimSegmentMin 12  --chimJunctionOverhangMin 8  --chimOutJunctionFormat 1  --alignSJDBoverhangMin 10  --alignMatesGapMax 100000  --alignIntronMax 100000  --alignSJstitchMismatchNmax 5 -1 5 5  --runThreadN 10 --outSAMstrandField intronMotif  --outSAMunmapped Within  --alignInsertionFlush Right  --alignSplicedMateMapLminOverLmate 0  --alignSplicedMateMapLmin 30  --outSAMtype BAM Unsorted  --readFilesIn /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusion_Quantify/SimilarWT_Fusion1_0.4.fq /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusion_Quantify/SimilarWT_Fusion2_0.4.fq  --outSAMattrRGline ID:GRPundef  --chimMultimapScoreRange 3  --chimScoreJunctionNonGTAG -4  --ch

CMD: ln -sf /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/SimilarWT_Fusion_0.4/star-fusion.preliminary/star-fusion.filter.intermediates_dir/star-fusion.post_blast_and_promiscuity_filter /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/SimilarWT_Fusion_0.4/star-fusion.preliminary/star-fusion.fusion_candidates.preliminary.filtered
* Running CMD: /home/jp/STAR-Fusion-v1.10.0/util/incorporate_FFPM_into_final_report.pl /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/SimilarWT_Fusion_0.4/star-fusion.preliminary/star-fusion.fusion_candidates.preliminary.filtered 12120 > /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/SimilarWT_Fusion_0.4/star-fusion.preliminary/star-fusion.fusion_candidates.preliminary.filtered.FFPM
* Running CMD: /home/jp/STAR-Fusion-v1.10.0/util/append_breakpoint_junction_info.pl /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/SimilarWT_Fusion_0.4/star-fusion.preliminary/star-fusion.fusion_candidates.preliminary.f


done.

* Running CMD: /home/jp/STAR-Fusion-v1.10.0/util/STAR-Fusion.predict  -J /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/SimilarWT_Fusion_0.3/star-fusion.preliminary/star-fusion.junction_breakpts_to_genes.txt.pass  -O /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/SimilarWT_Fusion_0.3/star-fusion.preliminary/star-fusion 
-parsing fusion evidence: /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/SimilarWT_Fusion_0.3/star-fusion.preliminary/star-fusion.junction_breakpts_to_genes.txt.pass
-outputting fusion candidates to file: /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/SimilarWT_Fusion_0.3/star-fusion.preliminary/star-fusion.fusion_candidates.preliminary
* Running CMD: /home/jp/STAR-Fusion-v1.10.0/util/STAR-Fusion.filter  --fusion_preds /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/SimilarWT_Fusion_0.3/star-fusion.preliminary/star-fusion.fusion_candidates.preliminary  -J Chimeric.out.junction  --tmpdir /tmp  --min_

CMD: mkdir -p /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/SimilarWT_Fusion_0.2/star-fusion.preliminary
* Running CMD: /home/jp/anaconda3/bin/STAR --genomeDir /home/jp/STAR-Fusion-v1.10.0/GRCh38_gencode_v37_CTAT_lib_Mar012021.plug-n-play/ctat_genome_lib_build_dir/ref_genome.fa.star.idx  --outReadsUnmapped None  --chimSegmentMin 12  --chimJunctionOverhangMin 8  --chimOutJunctionFormat 1  --alignSJDBoverhangMin 10  --alignMatesGapMax 100000  --alignIntronMax 100000  --alignSJstitchMismatchNmax 5 -1 5 5  --runThreadN 10 --outSAMstrandField intronMotif  --outSAMunmapped Within  --alignInsertionFlush Right  --alignSplicedMateMapLminOverLmate 0  --alignSplicedMateMapLmin 30  --outSAMtype BAM Unsorted  --readFilesIn /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusion_Quantify/SimilarWT_Fusion1_0.2.fq /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusion_Quantify/SimilarWT_Fusion2_0.2.fq  --outSAMattrRGline ID:GRPundef  --chimMultimapScoreRange 3  --chimScoreJunctionNonGTAG -4  --ch

CMD: ln -sf /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/SimilarWT_Fusion_0.2/star-fusion.preliminary/star-fusion.filter.intermediates_dir/star-fusion.post_blast_and_promiscuity_filter /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/SimilarWT_Fusion_0.2/star-fusion.preliminary/star-fusion.fusion_candidates.preliminary.filtered
* Running CMD: /home/jp/STAR-Fusion-v1.10.0/util/incorporate_FFPM_into_final_report.pl /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/SimilarWT_Fusion_0.2/star-fusion.preliminary/star-fusion.fusion_candidates.preliminary.filtered 6113 > /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/SimilarWT_Fusion_0.2/star-fusion.preliminary/star-fusion.fusion_candidates.preliminary.filtered.FFPM
* Running CMD: /home/jp/STAR-Fusion-v1.10.0/util/append_breakpoint_junction_info.pl /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/SimilarWT_Fusion_0.2/star-fusion.preliminary/star-fusion.fusion_candidates.preliminary.fi


done.

* Running CMD: /home/jp/STAR-Fusion-v1.10.0/util/STAR-Fusion.predict  -J /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/SimilarWT_Fusion_0.1/star-fusion.preliminary/star-fusion.junction_breakpts_to_genes.txt.pass  -O /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/SimilarWT_Fusion_0.1/star-fusion.preliminary/star-fusion 
-parsing fusion evidence: /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/SimilarWT_Fusion_0.1/star-fusion.preliminary/star-fusion.junction_breakpts_to_genes.txt.pass
-outputting fusion candidates to file: /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/SimilarWT_Fusion_0.1/star-fusion.preliminary/star-fusion.fusion_candidates.preliminary
* Running CMD: /home/jp/STAR-Fusion-v1.10.0/util/STAR-Fusion.filter  --fusion_preds /home/jp/ManualFusions/TMPRSS2-ERG/WildType_Fusions_Results/SimilarWT_Fusion_0.1/star-fusion.preliminary/star-fusion.fusion_candidates.preliminary  -J Chimeric.out.junction  --tmpdir /tmp  --min_

: 1

# QC fusion results

## High WT Low Fusion

In [62]:
echo "Non downsampled"
egrep "TMPRSS2|ERG" WildType_Fusions_Results/HighWT_LowFusion/star-fusion.fusion_predictions.abridged.tsv|cut -f 1,2,8,10

echo -e "\nDownsampled 50%"
egrep "TMPRSS2|ERG" WildType_Fusions_Results/HighWT_LowFusion_0.5/star-fusion.fusion_predictions.abridged.tsv|cut -f 1,2,8,10

echo -e "\nDownsampled 40%"
egrep "TMPRSS2|ERG" WildType_Fusions_Results/HighWT_LowFusion_0.4/star-fusion.fusion_predictions.abridged.tsv|cut -f 1,2,8,10

echo -e "\n Downsampled 30%"
egrep "TMPRSS2|ERG" WildType_Fusions_Results/HighWT_LowFusion_0.3/star-fusion.fusion_predictions.abridged.tsv|cut -f 1,2,8,10

echo -e "\n Downsampled 20%"
egrep "TMPRSS2|ERG" WildType_Fusions_Results/HighWT_LowFusion_0.2/star-fusion.fusion_predictions.abridged.tsv|cut -f 1,2,8,10

echo -e "\n Downsampled 10%"
egrep "TMPRSS2|ERG" WildType_Fusions_Results/HighWT_LowFusion_0.1/star-fusion.fusion_predictions.abridged.tsv|cut -f 1,2,8,10


Non downsampled
TMPRSS2--ERG	63	chr21:41498133:-	chr21:38423561:-
TMPRSS2--ERG	52	chr21:41494356:-	chr21:38445621:-

Downsampled 50%
TMPRSS2--ERG	32	chr21:41498133:-	chr21:38423561:-
TMPRSS2--ERG	29	chr21:41494356:-	chr21:38445621:-

Downsampled 40%
TMPRSS2--ERG	23	chr21:41498133:-	chr21:38423561:-
TMPRSS2--ERG	21	chr21:41494356:-	chr21:38445621:-

 Downsampled 30%
TMPRSS2--ERG	21	chr21:41498133:-	chr21:38423561:-
TMPRSS2--ERG	17	chr21:41494356:-	chr21:38445621:-

 Downsampled 20%
TMPRSS2--ERG	13	chr21:41494356:-	chr21:38445621:-
TMPRSS2--ERG	13	chr21:41498133:-	chr21:38423561:-

 Downsampled 10%
TMPRSS2--ERG	7	chr21:41498133:-	chr21:38423561:-
TMPRSS2--ERG	5	chr21:41494356:-	chr21:38445621:-


: 1

## Low WT High Fusion

In [65]:
echo "Non downsampled"
egrep "TMPRSS2|ERG" WildType_Fusions_Results/LowWT_HighFusion/star-fusion.fusion_predictions.abridged.tsv|cut -f 1,2,8,10

echo -e "\nDownsampled 50%"
egrep "TMPRSS2|ERG" WildType_Fusions_Results/LowWT_HighFusion_0.5/star-fusion.fusion_predictions.abridged.tsv|cut -f 1,2,8,10

echo -e "\nDownsampled 40%"
egrep "TMPRSS2|ERG" WildType_Fusions_Results/LowWT_HighFusion_0.4/star-fusion.fusion_predictions.abridged.tsv|cut -f 1,2,8,10

echo -e "\n Downsampled 30%"
egrep "TMPRSS2|ERG" WildType_Fusions_Results/LowWT_HighFusion_0.3/star-fusion.fusion_predictions.abridged.tsv|cut -f 1,2,8,10

echo -e "\n Downsampled 20%"
egrep "TMPRSS2|ERG" WildType_Fusions_Results/LowWT_HighFusion_0.2/star-fusion.fusion_predictions.abridged.tsv|cut -f 1,2,8,10

echo -e "\n Downsampled 10%"
egrep "TMPRSS2|ERG" WildType_Fusions_Results/LowWT_HighFusion_0.1/star-fusion.fusion_predictions.abridged.tsv|cut -f 1,2,8,10


Non downsampled
TMPRSS2--ERG	260	chr21:41498133:-	chr21:38423561:-
TMPRSS2--ERG	228	chr21:41494356:-	chr21:38445621:-

Downsampled 50%
TMPRSS2--ERG	135	chr21:41498133:-	chr21:38423561:-
TMPRSS2--ERG	108	chr21:41494356:-	chr21:38445621:-

Downsampled 40%
TMPRSS2--ERG	97	chr21:41498133:-	chr21:38423561:-
TMPRSS2--ERG	90	chr21:41494356:-	chr21:38445621:-

 Downsampled 30%
TMPRSS2--ERG	74	chr21:41498133:-	chr21:38423561:-
TMPRSS2--ERG	58	chr21:41494356:-	chr21:38445621:-

 Downsampled 20%
TMPRSS2--ERG	52	chr21:41498133:-	chr21:38423561:-
TMPRSS2--ERG	37	chr21:41494356:-	chr21:38445621:-

 Downsampled 10%
TMPRSS2--ERG	27	chr21:41498133:-	chr21:38423561:-
TMPRSS2--ERG	19	chr21:41494356:-	chr21:38445621:-


: 1

## Similar WT and Fusion

In [66]:
echo "Non downsampled"
egrep "TMPRSS2|ERG" WildType_Fusions_Results/SimilarWT_Fusion/star-fusion.fusion_predictions.abridged.tsv|cut -f 1,2,8,10

echo -e "\nDownsampled 50%"
egrep "TMPRSS2|ERG" WildType_Fusions_Results/SimilarWT_Fusion_0.5/star-fusion.fusion_predictions.abridged.tsv|cut -f 1,2,8,10

echo -e "\nDownsampled 40%"
egrep "TMPRSS2|ERG" WildType_Fusions_Results/SimilarWT_Fusion_0.4/star-fusion.fusion_predictions.abridged.tsv|cut -f 1,2,8,10

echo -e "\n Downsampled 30%"
egrep "TMPRSS2|ERG" WildType_Fusions_Results/SimilarWT_Fusion_0.3/star-fusion.fusion_predictions.abridged.tsv|cut -f 1,2,8,10

echo -e "\n Downsampled 20%"
egrep "TMPRSS2|ERG" WildType_Fusions_Results/SimilarWT_Fusion_0.2/star-fusion.fusion_predictions.abridged.tsv|cut -f 1,2,8,10

echo -e "\n Downsampled 10%"
egrep "TMPRSS2|ERG" WildType_Fusions_Results/SimilarWT_Fusion_0.1/star-fusion.fusion_predictions.abridged.tsv|cut -f 1,2,8,10


Non downsampled
TMPRSS2--ERG	122	chr21:41494356:-	chr21:38445621:-
TMPRSS2--ERG	110	chr21:41498133:-	chr21:38423561:-

Downsampled 50%
TMPRSS2--ERG	61	chr21:41498133:-	chr21:38423561:-
TMPRSS2--ERG	58	chr21:41494356:-	chr21:38445621:-

Downsampled 40%
TMPRSS2--ERG	52	chr21:41494356:-	chr21:38445621:-
TMPRSS2--ERG	51	chr21:41498133:-	chr21:38423561:-

 Downsampled 30%
TMPRSS2--ERG	38	chr21:41498133:-	chr21:38423561:-
TMPRSS2--ERG	34	chr21:41494356:-	chr21:38445621:-

 Downsampled 20%
TMPRSS2--ERG	23	chr21:41494356:-	chr21:38445621:-
TMPRSS2--ERG	21	chr21:41498133:-	chr21:38423561:-

 Downsampled 10%
TMPRSS2--ERG	14	chr21:41494356:-	chr21:38445621:-
TMPRSS2--ERG	9	chr21:41498133:-	chr21:38423561:-


: 1